In [1]:
%load_ext autoreload
%autoreload 2

import os.path as osp
import sys
from mmengine.config import Config, DictAction
from mmengine.runner import Runner

from mmdet3d.datasets import *
from torch.utils.data import DataLoader

sys.path.append('../tools')
from visualizer import ImageVisualizer, draw_monodetection_labels, draw_monodetection_results, draw_monolabels_from_config

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [3]:
config_file = '/home/javier/sensus-loci/sensus/configs/smoke/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d.py'
sample_id = 3
cfg = draw_monolabels_from_config(config_file, sample_id)


/home/javier/sensus-loci/mmdetection3d/mmdet3d/evaluation/functional/kitti_utils/eval.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_thresholds(scores: np.ndarray, num_gt, num_sample_pts=41):


10/25 17:36:47 - mmengine - INFO - ------------------------------
10/25 17:36:47 - mmengine - INFO - The length of the dataset: 5653
10/25 17:36:47 - mmengine - INFO - The number of instances per category in the dataset:
+------------+--------+
| category   | number |
+------------+--------+
| Pedestrian | 20266  |
| Cyclist    | 13853  |
| Car        | 84073  |
+------------+--------+
/home/javier/datasets/DAIR/single-infrastructure-side-mmdet/training/image_2/004041.png
[2183.375019, 0.0, 940.590363, 0.0, 0.0, 2329.297332, 567.568513, 0.0, 0.0, 0.0, 1.0, 0.0]
[1.5452029705047607, 1.878633975982666, 4.276515007019043]
[-16.586124420166016, -7.096008777618408, 63.56165313720703]
1.5575275421142578
[1.0067009925842285, 0.7107509970664978, 1.7090480327606201]
[19.81849479675293, -13.005849838256836, 88.87401580810547]
-3.1346018314361572
[1.4836070537567139, 1.9794689416885376, 4.39279317855835]
[1.0624860525131226, 3.2255666255950928, 19.241079330444336]
-1.5651224851608276
[1.459364056

In [5]:
train_data_cfg = cfg['train_dataloader']['dataset']
print(train_data_cfg)


TypeError: 'NoneType' object is not subscriptable

In [7]:
print(cfg)

None


In [19]:
# Obtener el tipo de conjunto de datos y luego eliminar el campo "type" de la configuración
dataset_type = train_data_cfg.pop("type")

# Usar globals() para obtener la clase y luego instanciarla
DatasetClass = globals()[dataset_type]
train_dataset = DatasetClass(**train_data_cfg)
train_dataset = KittiDataset(**train_data_cfg)


10/05 16:24:40 - mmengine - INFO - ------------------------------
10/05 16:24:40 - mmengine - INFO - The length of the dataset: 5653
10/05 16:24:40 - mmengine - INFO - The number of instances per category in the dataset:
+------------+--------+
| category   | number |
+------------+--------+
| Pedestrian | 20266  |
| Cyclist    | 13853  |
| Car        | 84073  |
+------------+--------+
10/05 16:24:49 - mmengine - INFO - ------------------------------
10/05 16:24:49 - mmengine - INFO - The length of the dataset: 5653
10/05 16:24:49 - mmengine - INFO - The number of instances per category in the dataset:
+------------+--------+
| category   | number |
+------------+--------+
| Pedestrian | 20266  |
| Cyclist    | 13853  |
| Car        | 84073  |
+------------+--------+


In [13]:
sample = train_dataset[0]  # This will give you the first item in the dataset.

In [14]:
img_path = sample['data_samples'].img_path

print(img_path)

/home/javier/datasets/DAIR/single-infrastructure-side-mmdet/training/image_2/000420.png


In [15]:
lidar_to_cam = sample['data_samples'].lidar2cam
cam_to_img = sample['data_samples'].cam2img
print(cam_to_img)
print(lidar_to_cam)

[[2183.375019, 0.0, 940.590363, 0.0], [0.0, 2329.297332, 567.568513, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]
[[-2.98502352e-02 -9.96147215e-01  1.45273365e-03 -2.28539348e+00]
 [-2.23098665e-01  2.12258496e-03 -8.56588423e-01  5.62775612e+00]
 [ 9.74099755e-01 -4.10278775e-02 -1.90818235e-01  1.41027522e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [16]:
gt_boxes_3d = sample['data_samples'].gt_instances_3d.bboxes_3d.tensor
print(gt_boxes_3d)

tensor([[ 4.6987e+00,  9.5145e-01,  2.8951e+01,  4.6676e+00,  1.4991e+00,
          1.8193e+00, -1.5293e+00],
        [-2.5179e+00, -1.4620e+01,  9.6542e+01,  4.2137e+00,  1.8276e+00,
          1.9293e+00, -1.5667e+00],
        [ 4.3949e+00, -2.9864e+01,  1.6471e+02,  4.3024e+00,  1.5420e+00,
          1.8503e+00, -1.5700e+00],
        [ 4.8977e-01, -2.4962e+01,  1.4133e+02,  4.3181e+00,  1.5729e+00,
          1.5752e+00, -1.5604e+00],
        [-1.1801e+01,  4.8713e-01,  2.9914e+01,  5.5835e-01,  1.5633e+00,
          5.2690e-01, -2.9515e+00],
        [-5.3893e+00,  2.8436e+00,  2.0108e+01,  4.1925e+00,  1.4915e+00,
          1.9711e+00, -1.5773e+00],
        [-2.3052e+01, -1.0694e+01,  7.9779e+01,  4.1110e+00,  1.6344e+00,
          1.8950e+00,  1.6070e+00],
        [-2.8183e+00, -2.3708e+01,  1.3609e+02,  4.3335e+00,  1.5319e+00,
          2.3326e+00, -1.5614e+00],
        [ 8.4806e-01, -3.6094e+01,  1.9096e+02,  2.6170e-01,  6.2944e-01,
          1.7493e+00, -1.5585e+00],
        [ 

In [16]:
print(sample)


{'data_samples': <Det3DDataSample(

    META INFORMATION
    affine_aug: True
    cam2img: [[2183.375019, 0.0, 979.409637, 0.0], [0.0, 2329.297332, 567.568513, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]
    transformation_3d_flow: ['HF']
    lidar2cam: array([[-2.98502352e-02, -9.96147215e-01,  1.45273365e-03,
                -2.28539348e+00],
               [-2.23098665e-01,  2.12258496e-03, -8.56588423e-01,
                 5.62775612e+00],
               [ 9.74099755e-01, -4.10278775e-02, -1.90818235e-01,
                 1.41027522e+00],
               [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                 1.00000000e+00]])
    box_type_3d: <class 'mmdet3d.structures.bbox_3d.cam_box3d.CameraInstance3DBoxes'>
    lidar2img: array([[ 8.51054585e+02, -2.21355337e+03, -1.76309931e+02,
                -3.66337974e+03],
               [ 3.32052234e+01, -1.83419999e+01, -2.10355155e+03,
                 1.39091451e+04],
               [ 9.74099755e-01, -4.10278775e-02